# 主な意見の発言者特定
- 文を入れると発言者を返す
- Few shotをプロンプトテンプレートで
- Gradioのインターフェース
- PDFを入れると発言を抽出し、発言者を特定する

In [37]:
!pip install openai

In [38]:
!pip install langchain

In [39]:
!pip install gradio

In [40]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

import pandas as pd
import gradio as gr

In [41]:
os.environ["OPENAI_API_KEY"] = 'xxx'
openai.api_key = os.getenv("OPENAI_API_KEY")

In [43]:
chat = ChatOpenAI(temperature=0)

In [44]:
fewshot = '''
１．先行きの物価上昇率の低下が見込まれ、海外経済の不確実性の高まりもみられる中では、現在の金融緩和を継続すべきである。 植田
２．２％の「物価安定の目標」の持続的達成のためには、賃金上昇を伴う形で実現する必要がある。今年の春季労使交渉では想定以上の賃上げが進む見通しであるが、名目賃金上昇率が物価対比でみて十分に高まるよう、金融緩和を通じて賃上げのモメンタムをしっかりと支え続けることが必要である。 内田
３．物価見通しは幾分上振れているが、「２％を超えるインフレ率が持続してしまうリスク」より、「拙速な金融緩和の修正によって２％実現の機会を逸してしまうリスク」の方がずっと大きい。 田村
'''

In [45]:
input_sentences = '''
    先行きの物価見通しなどを踏まえると、現在の金融緩和を継続することが適当である。
    ２％の「物価安定の目標」を持続的・安定的に実現するためには、コスト・プッシュによる物価上昇ではなく、賃金上昇を伴う物価上昇が必要である。
'''

In [46]:
template = '''
### 命令文
過去の発言をもとに、判定対象発言の発言者を推測してください。
また判断した理由も出力してください。
### 判定対象発言
{判定対象発言}
### 出力
表形式で出力してください。
ヘッダーは、[発言]、[発言者]、[判定理由] としてください。
### 過去の発言とその発言者
{過去発言}
'''

In [47]:
prompt = PromptTemplate(
		template=template,
    input_variables=["判定対象発言", "過去発言"]
)

In [48]:
prompt_text = prompt.format(判定対象発言=input_sentences, 過去発言=fewshot)

In [49]:
messages = [
    SystemMessage(content="あなたはに日銀の金融政策に精通したエコノミストです。"),
    HumanMessage(content=prompt_text)
]

In [50]:
response = chat(messages)

In [51]:
response.content

'| 発言 | 発言者 | 判定理由 |\n| --- | --- | --- |\n| 先行きの物価見通しなどを踏まえると、現在の金融緩和を継続することが適当である。 | 植田 | 「先行きの物価上昇率の低下が見込まれ、海外経済の不確実性の高まりもみられる中では、現在の金融緩和を継続すべきである」という発言と内容が一致している。 |\n| ２％の「物価安定の目標」を持続的・安定的に実現するためには、コスト・プッシュによる物価上昇ではなく、賃金上昇を伴う物価上昇が必要である。 | 内田 | 「２％の「物価安定の目標」の持続的達成のためには、賃金上昇を伴う形で実現する必要がある」という発言と内容が一致している。 |'

In [52]:
def read_csv(file):
    # Read the uploaded CSV file into a Pandas DataFrame
    df = pd.read_csv(file.name, encoding='shift_jis')
    return df

In [57]:
def speaker_estimation(file):
  df = read_csv(file)
  input_sentences = df['sentences']
  print(input_sentences)
  prompt_text = prompt.format(判定対象発言=input_sentences, 過去発言=fewshot)
  messages = [
      SystemMessage(content="あなたはに日銀の金融政策に精通したエコノミストです。"),
      HumanMessage(content=prompt_text)
  ]
  response = chat(messages)
  df_out = pd.DataFrame(response.content, index=['test'])
  return df_out

In [58]:
with gr.Blocks() as demo:
  gr.Markdown("""
  # 主な意見発言者推定
  """)
  with gr.Row():
    inp = gr.inputs.File(label="Upload a CSV file")
  with gr.Row():
    out = gr.DataFrame()
  inp.change(speaker_estimation, inp, out)

<ipython-input-58-05ec9cace944>:6: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inp = gr.inputs.File(label="Upload a CSV file")
<ipython-input-58-05ec9cace944>:6: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inp = gr.inputs.File(label="Upload a CSV file")
<ipython-input-58-05ec9cace944>:6: GradioDeprecationWarning: `keep_filename` parameter is deprecated, and it has no effect
  inp = gr.inputs.File(label="Upload a CSV file")


In [59]:
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
demo = gr.Interface(
    fn=speaker_estimation,
    inputs=gr.Textbox(placeholder="判定したい表現を入力してください。"),
    outputs="text",
    interpretation="default",
    examples=[["先行きの物価見通しなどを踏まえると、現在の金融緩和を継続することが適当である。"]])

In [ ]:
demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": Question},
    ],
)

In [ ]:
response.choices[0]["message"]["content"]

'| 発言 | 発言者 | 判定理由 |\n|------|--------|----------|\n| 先行きの物価見通しなどを踏まえると、現在の金融緩和を継続することが適当である。 | 植田 | 「先行きの物価上昇率の低下が見込まれ、海外経済の不確実性の高まりもみられる中では、現在の金融緩和を継続すべきである」という過去の発言と一致するため、発言者は植田と推測される。 |\n| ２％の「物価安定の目標」を持続的・安定的に実現するためには、コスト・プッシュによる物価上昇ではなく、賃金上昇を伴う物価上昇が必要である。 | 内田 | 「２％の「物価安定の目標」の持続的達成のためには、賃金上昇を伴う形で実現する必要がある」という過去の発言と一致するため、発言者は内田と推測される。 |'